the missing value is on AFRM , it's for the next quarter it's not labeled +1q that's the reason behind that.

In [3]:
import time as _tm
import requests as _rq
import pandas as _pd
import openpyxl
API_TOKEN = "67ffece4b2ae08.94077168"
BASE_URL  = "https://eodhd.com/api"
HEADERS   = {"User-Agent": "EPS-Trend/1.1"}
PAUSE_SEC = 1.1                     


def _get_json(url: str) -> dict:
    r = _rq.get(url, headers=HEADERS, timeout=30)
    r.raise_for_status()
    return r.json()


def _pull_trend(ticker: str) -> dict:

    url = (
        f"{BASE_URL}/fundamentals/{ticker}"
        f"?filter=Earnings::Trend"           
        f"&api_token={API_TOKEN}&fmt=json"
    )
    data = _get_json(url)

    if "Earnings" in data:          
        return data.get("Earnings", {}).get("Trend", {})
    return data                     


def _latest_snapshots(trend: dict) -> dict[str, dict]:
    latest = {}
    for date_str, rec in trend.items():
        p = rec.get("period")
        if not p:
            continue
        ts = _pd.to_datetime(date_str, errors="coerce")
        if p not in latest or ts > latest[p]["_ts"]:
            rec["_ts"] = ts
            latest[p] = rec
    return latest


def _row_for(ticker: str) -> dict:
    trend = _pull_trend(ticker)
    if not trend:
        return {"Ticker": ticker.split('.', 1)[0], "ERROR": "no Trend"}

    snap = _latest_snapshots(trend)

    periods = {
        "Curr Qtr": "0q",
        "Next Qtr": "+1q",
        "Curr Yr":  "0y",
        "Next Yr":  "+1y",
    }

    def val(rec, fld):
        return float(rec.get(fld)) if rec and rec.get(fld) not in (None, "") else None

    row = {"Ticker": ticker.split('.', 1)[0]}
    for label, code in periods.items():
        rec = snap.get(code, {})
        row[f"{label} Curr"] = val(rec, "epsTrendCurrent")
        row[f"{label} 7"]    = val(rec, "epsTrend7daysAgo")
        row[f"{label} 30"]   = val(rec, "epsTrend30daysAgo")
        row[f"{label} 60"]   = val(rec, "epsTrend60daysAgo")
        row[f"{label} 90"]   = val(rec, "epsTrend90daysAgo")
    return row


def build_eps_trends(tickers: list[str]) -> _pd.DataFrame:
    rows = []
    for tk in tickers:
        print(f"{tk} ...", end="", flush=True)
        try:
            rows.append(_row_for(tk))
            print("ok")
        except Exception as e:
            print(f"ERR: {e}")
            rows.append({"Ticker": tk.split('.', 1)[0], "ERROR": str(e)})
        _tm.sleep(PAUSE_SEC)
    return _pd.DataFrame(rows)


def save(df: _pd.DataFrame, fname="eps_trend.csv"):
    if fname.lower().endswith(".xlsx"):
        df.to_excel(fname, index=False, engine="openpyxl")
    else:
        df.to_csv(fname, index=False, encoding="utf-8")
    # print(f"\n✅  Saved {len(df)} rows → {fname}")




aapl.US ...ok
amzn.US ...ok
amd.US ...ok
goog.US ...ok

✅  Saved 4 rows → eps_trend.xlsx


In [4]:
# if __name__ == "__main__":
#     test = ['aapl', 'amzn', 'amd', 'goog']
#     raw = [
#         "NET", "CRS", "ORLY", "DOCN", "TEM", "VST", "CVS", "TER", "RIVN", "NTGR",
#         "SPOT", "UBER", "SNOW", "ACHR", "ASAN", "RKLB", "DPZ", "LYFT", "NEGG", "MSFT",
#         "AAPL", "ARM", "AMZN", "META", "NFLX", "NVDA", "GOOG", "RDFN", "MDB", "Z",
#         "INTC", "AMD", "AFRM", "COIN", "TSLA", "HOOD", "CAVA", "F", "GM", "SMR", "U",
#         "RDDT", "SHOP", "SOFI"
#     ]
#     tickers = [f"{t}.US" for t in test]
#     tbl = build_table(tickers)



,Ticker,Curr Qtr Curr,Curr Qtr 7,Curr Qtr 30,Curr Qtr 60,Curr Qtr 90,Next Qtr Curr,Next Qtr 7,Next Qtr 30,Next Qtr 60,...,Curr Yr Curr,Curr Yr 7,Curr Yr 30,Curr Yr 60,Curr Yr 90,Next Yr Curr,Next Yr 7,Next Yr 30,Next Yr 60,Next Yr 90
0,aapl,1.4146,1.4656,1.4929,1.4962,1.4964,NaN,NaN,NaN,NaN,...,7.1791,7.2465,7.3004,7.3159,7.3343,7.8483,7.9705,8.1704,8.1991,8.2163
1,amzn,1.3228,1.3933,1.4162,1.4159,1.4169,1.5048,1.5504,1.6006,1.5967,...,6.1936,6.2078,6.3247,6.3334,6.2280,7.2976,7.4375,7.5834,7.5954,7.6253
2,amd,0.9336,0.9412,0.9348,0.9356,0.9439,0.8815,0.9003,1.0077,1.0117,...,4.4021,4.4083,4.6523,4.6821,4.9201,5.9452,5.9581,6.2184,6.3089,6.8178
3,goog,2.1730,2.1386,2.1761,2.1772,2.1884,2.2436,2.2312,2.2707,2.2707,...,9.5499,8.8890,8.9767,8.9871,8.9965,10.1939,10.0514,10.2293,10.2395,10.2488
